In [ ]:
using Pkg; Pkg.activate(".")
using Random, Distributions, LinearAlgebra, Statistics
using Plots, Plots.PlotMeasures, StatsPlots, LaTeXStrings
using Zygote, Flux, Turing, MCMCChains, AdvancedMH, Optim, AdvancedHMC
using DistributionsAD, AdvancedVI, Distributions
using DiffEqFlux, DifferentialEquations, DiffEqCallbacks
using LinearAlgebra, SymPy, StatsBase
using AdaptiveMCMC, DataFrames, KernelDensity
using CSV, Trapz, QuadGK, DelimitedFiles, Optim

In [ ]:
times = readdlm("./k1m3h1/times.mat", Float32)
expTimes = vcat(times[1,1:end])
filteredTimes = filter(x -> 0.01<=x<=0.5, expTimes)
expTimes2 = filteredTimes

In [ ]:
indices = findall(x -> 0.01 <= x <= 0.5, expTimes)

In [ ]:
#Defining Constants
D = 0.0485
Df = 0.001

m=1.0613
mf=9.33

k = 26912.6
b = 1.70338
g0 =26.1782
kf = 180.714
bf = 0.176391

uInitial = [0.019893, 0.0200593, 0.00754123, 0.00832097]
u0=uInitial

tspan = (expTimes2[1], expTimes2[end])
storedTimes = expTimes2
inadType=1

params = [k,b,bf,g0,kf]
pTrue = vcat(params,u0)

# For high fidelity
ki = 25788
bi = 18.206
g0i =26
kfi = 722.0
bfi = 0.176

k1=40000
k2=5

mi=1.0613
mfi=9.33

paramsHigh =[ki,bi,bfi,g0i,kfi,k1,k2]
pTrueHigh=vcat(paramsHigh,u0)

In [ ]:
function lowFidelity(du,u,p,t)
    k = p[1]
    b = p[2]
    bf = p[3]
    g0 = p[4]
    kf = p[5]

    if inadType == 1 || inadType == 2
        x = u[1:2]
        v = u[3:4]
        
        du[1:2]=v
        
        du[3]=(1/m)*(-b*(v[1]-v[2])-g0*v[1]-k*(x[1]-x[2]))
        du[4]=(1/mf)*(b*(v[1]-v[2])+k*(x[1]-x[2])-bf*v[2]-kf*x[2])
    end
    return du
end

function highFidelity(du,u,p,t)
    ki = p[1]
    bi = p[2]
    bfi = p[3]
    g0i = p[4]
    kfi = p[5]

    k1 = p[6]
    k2 = p[7]

    if inadType == 3 || inadType == 4
        x = u[1:2]
        v = u[3:4]
        
        du[1:2]=v
        
        du[3]=(1/mi)*(-bi*(v[1]-v[2])-g0i*v[1]-(ki*(x[1]-x[2])+k1*abs(x[1]-x[2])*(x[1]-x[2])+k2*(x[1]-x[2])^3))
        du[4]=(1/mfi)*(bi*(v[1]-v[2])+ki*(x[1]-x[2])-bfi*v[2]-kfi*x[2])
    end
    return du
end

In [ ]:
function takeDerivs(p; fp=false)
    vals = SavedValues(Float64, Vector{Float64})
    cb = SavingCallback((u,t,integrator)->integrator(t,Val{1}), vals, saveat=storedTimes)
    lowFidel = ODEProblem(lowFidelity, p[6:9],tspan,p)
    sol = Array(solve(lowFidel,RadauIIA5(), u0=p[6:9], p=p,saveat=storedTimes, reltol=1e-4, abstol=1e-4, callback=cb))
        
    derivs = mapreduce(permutedims, vcat, vals.saveval)
    
    return sol, derivs
end

function takeDerivsInad(p; fp=false)
    vals = SavedValues(Float64, Vector{Float64})
    cb = SavingCallback((u,t,integrator)->integrator(t,Val{1}), vals, saveat=storedTimes)
    highFidel = ODEProblem(highFidelity, p[8:11],tspan,p)
    sol = Array(solve(highFidel,RadauIIA5(), u0=p[8:11], p=p,saveat=storedTimes, reltol=1e-4, abstol=1e-4, callback=cb))
        
    derivs = mapreduce(permutedims, vcat, vals.saveval)
    
    return sol, derivs
end

In [ ]:
#solution, derivatives = takeDerivs(pTrue)

In [ ]:
data_accm = readdlm("./k1m3h1/acc_output5.mat", Float32)
data_accm_pr = vcat(data_accm[1,1:end])
data_accmf = readdlm("./k1m3h1/acc_w_output5.mat", Float32)
data_accmf_pr = vcat(data_accm[1,1:end])

accm = data_accm_pr[104:2612, :]
accf = data_accmf_pr[104:2612, :]

In [ ]:
# # plot(storedTimes,accm)
# # plot!(storedTimes, derivatives[:,3])
# # xlims!(0,0.5)

# plot(storedTimes,accm, label ="experiment",title="m=2 and k=1", linewidth=1.5, color=:blue)
# plot!(storedTimes, derivatives[:,3], label="low fidelity", linestyle=:dash, linewidth=2, color=:red)
# xlabel!("Time")
# ylabel!("Acceleration")
# #savefig("RRLogNormalm2k1_not_recal.png")

In [ ]:
dfInad = CSV.read("logNormalMCMCChains_M1K1_No_Hyper_Trial6_200000_L5_Inad.csv", DataFrame)
df = CSV.read("logNormalMCMCChains_M1K1_No_Hyper_Trial3_300000_L5_Updated_Priors_2.csv", DataFrame)

In [ ]:
dfInad 

In [ ]:
sdf = df[1:200:end, :]
sdfInad = dfInad[1:200:end, :]

In [ ]:
nTrajInad=801
nTraj=1201

In [ ]:
scombinedpsInad = [ [sdfInad[:, :x1][i], sdfInad[:, :x2][i], sdfInad[:, :x3][i], sdfInad[:, :x4][i], sdfInad[:, :x5][i], sdfInad[:, :x6][i], sdfInad[:, :x7][i], sdfInad[:, :x8][i], sdfInad[:, :x9][i], sdfInad[:, :x10][i], sdfInad[:, :x11][i]] for i in 1:nTrajInad ]
scombinedps = [ [sdf[:, :x1][i], sdf[:, :x2][i], sdf[:, :x3][i], sdf[:, :x4][i], sdf[:, :x5][i], sdf[:, :x6][i], sdf[:, :x7][i], sdf[:, :x8][i], sdf[:, :x9][i]] for i in 1:nTraj ]


In [ ]:
inadType=1
MCMC_results=[]
for param in scombinedps
    pTrue = param
    
    solution, derivatives = takeDerivs(pTrue)
    
    push!(MCMC_results, (derivatives[:,3]))
end
    

In [ ]:
inadType=3
MCMC_resultsInad=[]
for param in scombinedpsInad
    pTrue = param
    
    solution, derivatives = takeDerivsInad(pTrue)
    
    push!(MCMC_resultsInad, (derivatives[:,3]))
end

In [ ]:
ytrajectories = [MCMC_results[i] for i in 1:nTraj]
trajectories = hcat(ytrajectories...)

quantilesToPlot = [0.25,0.75,0.025,0.975]
function computeQuantiles(data, quantiles)
    return [quantile(data, q) for q in quantiles]
end

quantile_data = mapslices(data -> computeQuantiles(data, quantilesToPlot), trajectories, dims=2)

quantile50L = quantile_data[:, 1]
quantile50U = quantile_data[:, 2]
quantile95L = quantile_data[:, 3]
quantile95U = quantile_data[:, 4]

plot(storedTimes, quantile50L, label=nothing, color=:blue)
plot!(storedTimes, quantile50U, label=nothing, color=:blue)
plot!(storedTimes, quantile95L, label=nothing, color=:red)
plot!(storedTimes, quantile95U, label=nothing, color=:red)

In [ ]:
saverage_trajectory = mean(trajectories, dims=2)
savg = vec(saverage_trajectory)

In [ ]:
fill50=plot(storedTimes,quantile50L,fillrange=quantile50U, fillalpha=0.75, fillcolor=:orange,color=:orange, label=nothing)
fill95=plot(storedTimes,quantile95L,fillrange=quantile95U, fillalpha=1.0, fillcolor=:orange,color=:orange, label=nothing)

p4=plot(fill50)
p4=plot!(fill95)
p4=plot!(storedTimes,accm, label ="experiment", linewidth=1.5, color=:blue)
p4=plot!(storedTimes, savg, label="calibrated", linestyle=:dash, linewidth=2, color=:red)
xlabel!("Time")
ylabel!("Acceleration")
#savefig("calibrated_lognormal_m3k1_graph.png")

In [ ]:
seedNum=1000
function predictiveAssessment(enriched, observed; alpha =0.05)
    numObs = length(observed)
    gammas = zeros(numObs)
    numSamples = (size(enriched)[2])
    Random.seed!(seedNum)
    numObsError = 100
    
    predictions = zeros(numSamples, numObs)

    postPred = zeros(numObs, numSamples*numObsError)
    
    for j in 1:numObs      
        for k in 1:numSamples
            mu = enriched[j,k]
            postPred[j,(k-1)*numObsError+1:(k-1)*numObsError+numObsError]= rand(Normal(mu,0.1),numObsError)
        end
        density = kde(postPred[j,:])
        ik=InterpKDE(density)
        minf = minimum(postPred[j,:])
        maxf = maximum(postPred[j,:])
        xs = LinRange(minf,maxf,200)
        fpdf = pdf(ik,xs)
        fpdfLen = length(fpdf)
        pofy=pdf(ik,observed[j])
            
        for l in 1:fpdfLen
            if fpdf[l]>pofy
                fpdf[l]=0.
            end
        end
        gamma = -trapz(fpdf,xs)
        gammas[j]=gamma
    end
    return gammas, postPred
    
end

function fractionGamma(gamma;alpha=0.05)
    numObs = length(gamma)
    tau = alpha/numObs
    count = 0
    for i in 1:numObs
        if gamma[i]<tau
            count+=1
        end
    end
    count/numObs
end

In [ ]:
Gs=predictiveAssessment(trajectories,accm)

In [ ]:
GammaVals = Gs[1]
posteriors=Gs[2]

In [ ]:
fractionGamma(GammaVals)

In [ ]:
ytrajectoriesInad = [MCMC_resultsInad[i] for i in 1:nTrajInad]
trajectoriesInad = hcat(ytrajectoriesInad...)

quantile_dataInad = mapslices(data -> computeQuantiles(data, quantilesToPlot), trajectoriesInad, dims=2)

quantile50LInad = quantile_dataInad[:, 1]
quantile50UInad = quantile_dataInad[:, 2]
quantile95LInad = quantile_dataInad[:, 3]
quantile95UInad = quantile_dataInad[:, 4]

plot(storedTimes, quantile50LInad, label=nothing, color=:blue)
plot!(storedTimes, quantile50UInad, label=nothing, color=:blue)
plot!(storedTimes, quantile95LInad, label=nothing, color=:red)
plot!(storedTimes, quantile95UInad, label=nothing, color=:red)

In [ ]:
average_trajectoryInad = mean(trajectoriesInad, dims=2)
avgInad = vec(average_trajectoryInad)

In [ ]:
fill50Inad=plot(storedTimes,quantile50LInad,fillrange=quantile50UInad, fillalpha=0.75, fillcolor=:orange,color=:orange, label=nothing)
fill95Inad=plot(storedTimes,quantile95LInad,fillrange=quantile95UInad, fillalpha=1.0, fillcolor=:orange,color=:orange, label=nothing)

p3=plot(fill50Inad)
p3=plot!(fill95Inad)
p3=plot!(storedTimes,accm, label ="experiment", linewidth=1.5, color=:blue)
p3=plot!(storedTimes, avgInad, label="enriched", linestyle=:dash, linewidth=2, color=:red)
xlabel!("Time")
ylabel!("Acceleration")
#savefig("enriched_lognormal_m3k1_graph.png")

In [ ]:
GsInad=predictiveAssessment(trajectoriesInad,accm)

In [ ]:
GammaValsInad = GsInad[1]
posteriorsInad=GsInad[2]

In [ ]:
fractionGamma(GammaValsInad)